# Template - Strategy - Backtesting 

### Import Library

In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas_ta as ta
from backtesting.backtesting import Backtest, Strategy
from backtesting._plotting import set_bokeh_output
set_bokeh_output(notebook=False)

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 120
import warnings
warnings.filterwarnings('ignore')

/home/zuongthao/PycharmProjects/algo-stock/venv/lib/python3.11/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

### Load Price Data

In [2]:
import os
from pathlib import Path
notebook_path = os.getcwd()
algo_dir = Path(notebook_path).parent.parent
csv_file = str(algo_dir) + '/vn-stock-data/VN30ps/VN30F1M_5minutes.csv'
is_file = os.path.isfile(csv_file)
if is_file:
    dataset = pd.read_csv(csv_file, index_col='Date', parse_dates=True)
else:
    print('remote')
    dataset = pd.read_csv("https://raw.githubusercontent.com/zuongthaotn/vn-stock-data/main/VN30ps/VN30F1M_5minutes.csv", index_col='Date', parse_dates=True)

In [3]:
data = dataset.copy()

In [4]:
# data = data[(data.index > '2020-11-01 00:00:00') & (data.index < '2024-10-01 00:00:00')]
data = data[data.index > '2020-11-01 00:00:00']

In [5]:
data

,Open,High,Low,Close,Volume
Date,,,,,
2020-11-02 09:00:00,900.1,900.2,899.3,900.1,1910
2020-11-02 09:05:00,900.2,900.2,898.7,899.4,1670
2020-11-02 09:10:00,899.5,900.0,899.0,899.5,1329
2020-11-02 09:15:00,899.4,899.5,898.2,898.6,1722
2020-11-02 09:20:00,898.5,898.6,896.5,898.2,2939
...,...,...,...,...,...
2025-02-13 14:15:00,1333.5,1335.4,1333.4,1335.0,5583
2025-02-13 14:20:00,1335.4,1336.5,1334.7,1334.9,5753
2025-02-13 14:25:00,1335.3,1336.8,1334.9,1336.8,4879


In [10]:
df = data.copy()
# Calculate ATR (for wick size detection)
df["atr"] = ta.atr(df["High"], df["Low"], df["Close"], length=14)

# Define resistance and support levels (example: pivot points or swing highs/lows)
df["pivot_high"] = df["High"].shift(1).rolling(window=10).max()
df["pivot_low"] = df["Low"].shift(1).rolling(window=10).min()

# Bull Trap Detection
df["bull_trap"] = (df["High"] > df["pivot_high"]) & (df["Close"] < df["pivot_high"]) & (df["High"] - df["Close"] > df["atr"] * 0.5)

# Bear Trap Detection
df["bear_trap"] = (df["Low"] < df["pivot_low"]) & (df["Close"] > df["pivot_low"]) & (df["Close"] - df["Low"] > df["atr"] * 0.5)

# Filter signals
bull_traps = df[df["bull_trap"]]
bear_traps = df[df["bear_trap"]]

In [11]:
bear_traps

,Open,High,Low,Close,Volume,atr,pivot_high,pivot_low,bull_trap,bear_trap
Date,,,,,,,,,,
2020-11-02 11:00:00,897.3,897.4,895.4,897.0,3418,1.545652,901.5,896.7,False,True
2020-11-03 11:25:00,908.1,908.2,906.6,908.0,4298,1.284235,910.8,907.6,False,True
2020-11-03 13:05:00,907.4,907.9,906.3,907.5,2081,1.254260,910.8,906.6,False,True
2020-11-03 13:10:00,907.5,907.8,906.0,907.3,2493,1.293319,910.8,906.3,False,True
2020-11-03 13:15:00,907.3,907.3,905.5,906.7,2406,1.329577,910.8,906.0,False,True
...,...,...,...,...,...,...,...,...,...,...
2025-02-10 09:45:00,1327.2,1327.8,1325.2,1327.5,7722,2.153919,1343.3,1326.3,False,True
2025-02-11 14:15:00,1329.7,1330.7,1328.9,1330.5,5322,1.311015,1333.0,1329.8,False,True
2025-02-12 13:15:00,1338.8,1339.0,1338.3,1339.0,1572,0.765708,1340.0,1338.6,False,True


In [13]:
bull_traps.tail(20)

,Open,High,Low,Close,Volume,atr,pivot_high,pivot_low,bull_trap,bear_trap
Date,,,,,,,,,,
2025-02-06 10:00:00,1344.3,1344.9,1342.8,1342.8,4003,1.563952,1344.3,1339.7,True,False
2025-02-06 14:10:00,1343.6,1343.7,1341.2,1341.5,5394,1.474367,1343.5,1337.2,True,False
2025-02-07 10:05:00,1339.2,1340.0,1338.1,1338.5,4613,1.571806,1339.6,1335.0,True,False
2025-02-07 10:30:00,1339.4,1340.2,1338.6,1338.8,2536,1.411244,1340.0,1336.2,True,False
2025-02-07 11:00:00,1340.3,1340.3,1339.1,1339.5,1955,1.300802,1340.2,1337.6,True,False
2025-02-07 11:10:00,1339.2,1340.9,1339.0,1340.2,2790,1.330283,1340.3,1337.6,True,False
2025-02-07 13:30:00,1341.6,1342.7,1341.2,1342.0,3548,1.329235,1342.2,1337.8,True,False
2025-02-10 13:15:00,1335.5,1336.4,1335.1,1335.6,4019,1.583007,1336.0,1331.8,True,False
2025-02-10 13:20:00,1335.6,1336.9,1335.4,1335.7,3264,1.577078,1336.4,1331.8,True,False


In [ ]:
def cal_signal(r):
    signal = ''
    if r['Close'] > r['Close_s1']:
        signal = 'long'
    elif r['Close'] > r['Close_s1']:
        signal = 'long'
    return signal

def prepare_data(data):
    data['Close_s1'] = data['Close'].shift(1)
    data['signal'] = data.apply(lambda r: cal_signal(r), axis=1)
    return data

In [ ]:
class MainStrategy(Strategy):
    def init(self):
        self._broker._cash = 1500
        super().init()

    def next(self):
        super().next()
        _time = self.data.index
        current_time = _time[-1]
        if current_time.hour == 14 and current_time.minute >= 25:
            if self.position.is_long or self.position.is_short:
                self.position.close()
            return

        if self.position:
            return 
        signal = self.data.signal[-1]
        close_price = self.data.Close[-1]
        open_price = self.data.Open[-1]
        if signal == 'long':
            buy_price = close_price
            sl = buy_price - 3
            tp = buy_price + 6
            self.buy(size=1, sl=sl, tp=tp, limit=buy_price)
        elif signal == 'short':
            sell_price = close_price
            sl = sell_price + 3
            tp = sell_price - 6
            self.sell(size=1, sl=sl, tp=tp, limit=sell_price)

In [ ]:
prepared_data = prepare_data(data)
prepared_data.dropna(inplace=True)

In [ ]:
bt = Backtest(prepared_data, MainStrategy, commission=.0003, exclusive_orders=True)
stats = bt.run()

In [ ]:
stats

In [ ]:
stats['_trades']

In [ ]:
# bt.plot()